# Import Package, and Func.

In [27]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit

from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli



from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile

In [28]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 
def FCI(dist):
    # 1. 분자 정의
    mol = gto.M(
        atom = 'H 0 0 0; H 0 0 {}'.format(dist),  # 수소 원자 2개, 거리 0.74 Å
        basis = 'sto-3g',              # 간단한 기저함수
        unit = 'Angstrom',
        spin = 0,                      # 전자 수 = 2, 총 spin = 0 (singlet)
        charge = 0
    )

    # 2. Hartree-Fock 계산
    mf = scf.RHF(mol)
    hf_energy = mf.kernel()

    # 3. FCI 계산 (Full CI)
    cisolver = fci.FCI(mol, mf.mo_coeff)
    fci_energy, fci_vector = cisolver.kernel()

    return fci_energy


def pad_bitstrings(prob_dict: dict) -> dict:
    """
    확률 딕셔너리의 비트스트링 key들을 0-padding해서 정렬된 형태로 변환

    Args:
        prob_dict (dict): 예: {'0': 0.25, '1': 0.25, '10': 0.25, '11': 0.25}

    Returns:
        dict: {'00': 0.25, '01': 0.25, '10': 0.25, '11': 0.25}
    """
    # 가장 긴 비트스트링의 길이를 기준으로 패딩
    max_len = max(len(k) for k in prob_dict)
    new_dict = {k.zfill(max_len): v for k, v in prob_dict.items()}
    return new_dict

def expectation_from_Z(prob_dict: dict, pauli_string: str) -> float:
    """
    확률 딕셔너리와 Pauli 문자열(I와 Z만 포함)을 받아 기대값을 계산
    Args:
        prob_dict (dict): {'00': 0.25, '01': 0.25, ...} (모든 키는 같은 길이의 비트스트링이어야 함)
        pauli_string (str): 예: 'IIZZ'

    Returns:
        float: 기대값
    """
    n = len(pauli_string)
    assert all(len(k) == n for k in prob_dict), "모든 비트스트링의 길이는 Pauli 문자열과 같아야 함"

    expectation = 0.0

    for bitstring, prob in prob_dict.items():
        eigenvalue = 1
        for idx, pauli_op in enumerate(pauli_string):
            if pauli_op == 'I':
                continue
            else :
                bit = int(bitstring[idx])  # 왼쪽이 MSB라고 가정
                eigenvalue *= 1 if bit == 0 else -1

        expectation += eigenvalue * prob

    return expectation.real

H = HGate()
Sdg = SdgGate()
pauli_I = Pauli("I")
pauli_X = Pauli("X")
pauli_Y = Pauli("Y")
pauli_Z = Pauli("Z")
energy_arr = [] 

def cost_func(parameter, ansatz, pauli_basis, coeffs, backend):
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    Energy = 0

    
    for i in range(len(pauli_basis)):
        ansatz_var = ansatz.copy()
        qc = ansatz_var.assign_parameters(param_dict)
        pauli = pauli_basis[i]
        coeff = coeffs[i]

        for idx, pauli_op in enumerate(pauli):
            if pauli_op == pauli_X : 
                qc.append(H, [idx])
            if pauli_op == pauli_Y : 
                qc.append(Sdg, [idx])
                qc.append(H, [idx])

        #ansatz_transpiled = transpile(qc, basis_gates=['rx', 'rz', 'cx'])  # IonQ 기준
        qc.measure_all()
        job = backend.run(qc, shots=2000)
        result = job.get_probabilities()
        # result= pad_bitstrings(raw_prob)
        #print(result)

        E_iteration = coeff * expectation_from_Z(result, pauli)
        #print(E_iteration)
        Energy += E_iteration
    
    energy_arr.append(Energy + repulsion)
    print(Energy)
    return Energy

In [ ]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
simulator_backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")


In [ ]:
atoms = ["H", "H"]
basis = 'sto3g'
dist = 0.735
coords = [(0,0,0), (dist,0,0)]
charge = 0
multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_spatial_orbitals = E_problem.num_spatial_orbitals

init_state = HartreeFock(num_spatial_orbitals,num_particles,mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)
#ansatz = ansatz_v.decompose().decompose().decompose().decompose()
pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)

res = minimize(
    cost_func,
    x0,
    args=(ansatz, pauli_basis, coeffs, simulator_backend),
    method="cobyla",
    options={"maxiter": 200} ,
)
